# Busca Cega

In [53]:
from pprint import pprint

In [112]:
class Busca:
    def __init__(self):
        pass
    
    def buscar(self, problema):
        raise NotImplementedError

    def solucao(self, no, problema):
        p = no
        while p:
            print p.acao
            p = p.pai
            
        raise NotImplementedError

class NoBusca(object):
    def __init__(self, problema, estado, pai, acao, custo_total):
        self.estado = estado   
        self.pai = pai
        self.acao = acao
        self.custo_total = custo_total
    
    @classmethod
    def from_pai(cls, problema, pai, acao):
        estado = problema.resultado(pai.estado, acao)
        custo_total = pai.custo_total + problema.custo_passo(pai.estado, acao)
        return cls(problema, estado, pai, acao, custo_total)
    
    @classmethod
    def from_init(cls, problema):
        return cls(problema, problema.estado_inicial, None, None, 0)

class Estado(object):
    def __init__(self, chave):
        self.chave = chave

class Acao(object):
    def __init__(self, chave):
        self.chave = chave
        
class Problema:
    estado_inicial = None
    estado_final = None
    
    def __init__(self):
        pass
    
    def custo_passo(self, estado, acao):
        raise NotImplementedError
        
    def resultado(self, estado, acao):
        raise NotImplementedError
        
    def acoes(self, estado):
        raise NotImplementedError
    
    def objetivo(self, estado):
        raise NotImplementedError

class EstadoRota(Estado):
    def __init__(self, chave):
        super(EstadoRota, self).__init__(chave)
        
    def __repr__(self):
        return ("Em(%s)" % str.capitalize(self.chave))
    
    def __str__(self):
        return self.__repr__()

class AcaoIrRota(Acao):
    def __init__(self, chave, destino, custo):
        super(AcaoIrRota, self).__init__(chave)
        self.destino = destino
        self.custo = custo
        
    def __repr__(self):
        return ("Ir(%s,%d)" % (str.capitalize(self.chave), self.custo))
    
    def __str__(self):
        return self.__repr__()

In [115]:
class RotasRomenia(Problema):
    def __init__(self):
        pass
        
    @classmethod
    def from_arquivo(cls, arquivo):
        with open(arquivo,'r') as f:
            estradas = f.readlines()
        estradas = map(str.strip,estradas)
        estradas = map(str.split,estradas,[','] * len(estradas))
        cls.estado_inicial = EstadoRota(estradas[0][0])
        cls.estado_final = EstadoRota(estradas[1][0])
        
        estados = {cls.estado_inicial.chave : (cls.estado_inicial, dict()), cls.estado_final.chave : (cls.estado_final, dict())}
        
        for caminho in estradas[2:]:
            #print caminho
            if estados.has_key(caminho[0]):
                origem = estados[caminho[0]]
            else:
                estados[caminho[0]] = (EstadoRota(caminho[0]), dict())
                origem = estados[caminho[0]]
            
            if estados.has_key(caminho[1]):
                destino = estados[caminho[1]]
            else:
                estados[caminho[1]] = (EstadoRota(caminho[1]), dict())
                destino = estados[caminho[1]]
            
            custo = int(caminho[2])
            
            acao = AcaoIrRota(destino[0].chave, destino, custo)
            origem[1][destino[0].chave] = acao
                
        cls.estados = estados         
            
        return cls()
        
    def objetivo(self, estado):
        return True if estado.chave == self.estado_final.chave else False
        
    def acoes(self, estado):
        return self.estados[estado.chave][1]
        
    def resultado(self, estado, acao):
        return self.estados[estado.chave][1][acao].destino[0]
    
    def custo_passo(self, estado, acao):
        return self.estados[estado.chave][1][acao].custo
        
class BuscaLargura(Busca):
    
    def buscar(self, problema):
        no = NoBusca.from_init(problema)
        if problema.objetivo(no.estado):
            return self.solucao(no)
        fronteira = [no]
        explorado = dict()
        while True:
            if len(fronteira) == 0:
                return False
            no = fronteira.pop(0)
            explorado[no.estado.chave] = no.estado
            print problema.acoes(no.estado)
            for acao in problema.acoes(no.estado):
                filho = NoBusca.from_pai(problema, no, acao)
                #print(filho)
                #pprint(explorado)
                if (filho not in explorado) and \
                    (not len(filter(lambda x: x.estado.chave == filho.estado.chave, fronteira)) > 1):
                    if problema.objetivo(filho.estado):
                        return self.solucao(filho)
                    fronteira.append(filho)
            
        
        
p = RotasRomenia.from_arquivo('romenia.txt')  
print p.estado_final.chave

b = BuscaLargura()

b.buscar(p)

bucharest
{'zerind': Ir(Zerind,75), 'timisoara': Ir(Timisoara,118), 'sibiu': Ir(Sibiu,140)}
{'oradea': Ir(Oradea,71), 'arad': Ir(Arad,75)}
{'lugoj': Ir(Lugoj,111), 'arad': Ir(Arad,118)}
{'oradea': Ir(Oradea,151), 'fagaras': Ir(Fagaras,99), 'rimnicu vilcea': Ir(Rimnicu vilcea,80), 'arad': Ir(Arad,140)}
{'zerind': Ir(Zerind,71), 'sibiu': Ir(Sibiu,151)}
{'zerind': Ir(Zerind,75), 'timisoara': Ir(Timisoara,118), 'sibiu': Ir(Sibiu,140)}
{'timisoara': Ir(Timisoara,111), 'mehadia': Ir(Mehadia,70)}
{'zerind': Ir(Zerind,75), 'timisoara': Ir(Timisoara,118), 'sibiu': Ir(Sibiu,140)}
{'zerind': Ir(Zerind,71), 'sibiu': Ir(Sibiu,151)}
{'bucharest': Ir(Bucharest,211), 'sibiu': Ir(Sibiu,99)}
bucharest
fagaras
sibiu
None


NotImplementedError: 